# Support Bot

Over the last few notebooks, you've assembled a powerful toolkit for automation. You've mastered creating and managing asynchronous tasks, handling file uploads, building event-driven workflows with webhooks, and integrating it all into a conversational Slack bot.

Now, it's time to combine these skills to build a practical, end-to-end solution that solves a real-world business problem: automating employee vacation claims.

In this notebook, we will create an intelligent support bot that lives in Slack. 

Employees will be able to submit their vacation expense claims by simply uploading a receipt and mentioning the bot. From there, our system will take over, using Manus to intelligently process the document and automatically add a structured entry into a central Notion database for the finance team to review.



## Supporting File Uplaods

Now that we have our basic Slack integration working, let's enhance it to handle file attachments. When employees submit support requests or vacation claims, they often need to include supporting documents like receipts, medical certificates, or authorization forms.

### Understanding Slack File Events

When a user mentions your bot with file attachments, Slack sends additional file information in the webhook payload. Let's first examine the structure of these events and then modify our server to process them.



In [1]:
import json

# Example of what a file attachment event looks like
file_event_example = {
    "type": "event_callback",
    "event": {
        "type": "app_mention",
        "user": "U1234567890",
        "text": "<@U09U5R59FDL> I need to submit my vacation claim with attached receipt",
        "ts": "1678886400.123456",
        "channel": "C09U5STV0FQ",
        "files": [
            {
                "id": "F1234567890",
                "name": "vacation_receipt.pdf",
                "mimetype": "application/pdf",
                "url_private": "https://files.slack.com/files-pri/T1234567890-F1234567890/vacation_receipt.pdf",
                "size": 123456
            }
        ]
    }
}

print("File event structure:")
print(json.dumps(file_event_example, indent=2))

File event structure:
{
  "type": "event_callback",
  "event": {
    "type": "app_mention",
    "user": "U1234567890",
    "text": "<@U09U5R59FDL> I need to submit my vacation claim with attached receipt",
    "ts": "1678886400.123456",
    "channel": "C09U5STV0FQ",
    "files": [
      {
        "id": "F1234567890",
        "name": "vacation_receipt.pdf",
        "mimetype": "application/pdf",
        "url_private": "https://files.slack.com/files-pri/T1234567890-F1234567890/vacation_receipt.pdf",
        "size": 123456
      }
    ]
  }
}


In this section, we will upgrade our bot to handle file attachments, creating a seamless workflow where employees can simply upload a file and mention the bot to kick off the entire process.

Let's start by adding a simple function here to upload our files to the Manus API.

```py
class FileIDAttachment(TypedDict):
    """Defines the structure for a File ID attachment for the Manus API."""
    file_id: str
    filename: str
    
import requests
import os
def upload_file_to_manus(file_content: bytes, filename: str) -> str:
    """
    Uploads a file to the Manus workspace and returns its file_id.
    """
    api_key = os.environ["MANUS_API_KEY"]
    base_url = "https://api.manus.ai/v1"

    # Step 1: Create a file record and get a presigned upload URL
    headers = {"API_KEY": api_key, "Content-Type": "application/json"}
    create_file_response = requests.post(
        f"{base_url}/files",
        headers=headers,
        json={"filename": filename}
    )
    create_file_response.raise_for_status()
    file_record = create_file_response.json()
    
    file_id = file_record["id"]
    upload_url = file_record["upload_url"]
    print(f"✓ File record created for '{filename}'. File ID: {file_id}")

    # Step 2: Upload the file content to the presigned URL
    upload_response = requests.put(
        upload_url,
        data=file_content,
        headers={"Content-Type": "application/octet-stream"} # Use octet-stream for raw bytes
    )
    upload_response.raise_for_status()
    
    print(f"✓ File '{filename}' uploaded successfully!")
    return file_id
```

Finally, let's integrate this into our main handle_slack_message function. The new logic checks the Slack event for a files array. If files are present, it downloads each one and uses our new helper to upload it to Manus, collecting the file_ids to attach to our task.

```py
def handle_slack_message(event: AppMentionEvent) -> None:
    """
    Handles Slack mentions, now with logic to process file attachments via the Files API.
    """
    channel_id = event["channel"]
    thread_id = event.get("thread_ts", event["ts"])
    
    raw_text = event.get("text", "")
    prompt_text = re.sub(r"^<@[\w\d]+>\s*", "", raw_text)

    client = get_slack_client()
    
    existing_task_id = thread_task_map.get(thread_id)
    
    
    manus_attachments: List[FileIDAttachment] = []
    if "files" in event:
        print(f"Found {len(event['files'])} files in the Slack message.")
        for file in event["files"]:
            file_url = file["url_private_download"]
            file_name = file["name"]
            
            # Download the file using the authenticated Slack URL
            response = requests.get(
                file_url,
                headers={"Authorization": f"Bearer {os.environ['SLACK_BOT_TOKEN']}"}
            )
            response.raise_for_status()
            
            # Upload the file to Manus and get the file_id
            file_id = upload_file_to_manus(response.content, file_name)
            
            # Prepare the attachment for the Manus task
            manus_attachments.append({
                "file_id": file_id,
                "filename": file_name,
            })

    if existing_task_id:
        print(f"Continuing existing task {existing_task_id} for thread {thread_id}")
        create_manus_task(prompt_text, task_id=existing_task_id,attachments=manus_attachments)
        
        client.reactions_add(
            channel=channel_id,
            name="eyes",
            timestamp=event["ts"]
        )
        return
        
    # --- Create the task with the prompt and any attachments ---
    manus_task = create_manus_task(prompt_text, attachments=manus_attachments)
    new_task_id = manus_task.get("task_id")
    # ... (rest of the function remains the same)
```

With this new change, you should be able to respond to the user's message, upload the file to Manus, and then create a new task with the prompt and attachments.


Our bot can now receive and understand vacation claims submitted via Slack. But what happens next? We'll need to create a system to process these claims and notify the appropriate team members.
